In [2]:
import torch

CHECKPOINT_PATH = "./checkpoints_2/last_checkpoint.pth"
state = torch.load(CHECKPOINT_PATH, map_location='cpu')
print(state.keys())

dict_keys(['0_DoubleConv_sub0', '0_DoubleConv_sub1', '1_Conv2d', '2_DoubleConv_sub0', '2_DoubleConv_sub1', '3_Conv2d', '4_DoubleConv_sub0', '4_DoubleConv_sub1', '5_Conv2d', '6_DoubleConv_sub0', '6_DoubleConv_sub1', '7_Conv2d', '8_DoubleConv_sub0', '8_DoubleConv_sub1', '9_ConvTransposed2d', '10_DoubleConv_sub0', '10_DoubleConv_sub1', '11_ConvTransposed2d', '12_DoubleConv_sub0', '12_DoubleConv_sub1', '13_ConvTransposed2d', '14_DoubleConv_sub0', '14_DoubleConv_sub1', '15_ConvTransposed2d', '16_DoubleConv_sub0', '16_DoubleConv_sub1', '17_Conv2d'])


/var/folders/d6/klnztyj51h5d3y23pmnjp7v00000gn/T/ipykernel_67839/419973279.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(CHECKPOINT_PATH, map_locati

In [7]:
import os

def load_custom_weights(model, checkpoint_path):
    print(f"가중치 파일 로드 중: {checkpoint_path}")
    if not os.path.exists(checkpoint_path):
        print("파일이 존재하지 않습니다.")
        return

    checkpoint = torch.load(checkpoint_path, map_location="cpu")

    for key, state_dict in checkpoint.items():
        try:
            parts = key.split("_")
            idx = int(parts[0])  # 모듈 인덱스
            if idx >= len(model.modules):
                continue

            target = model.modules[idx]  # enc1, down1, ...

            # DoubleConv 의 sub0/sub1 처리
            if len(parts) >= 3 and parts[1] == "DoubleConv":
                sub = parts[2]
                if sub == "sub0":
                    target = target.conv1
                elif sub == "sub1":
                    target = target.conv2

            # Conv2d, ConvTransposed2d 는 그대로 target 사용

            # 실제 가중치 주입
            for param_name, param_tensor in state_dict.items():
                if hasattr(target, param_name):
                    # 필요하면 clone() 해서 복사
                    setattr(target, param_name, param_tensor.clone())

        except Exception as e:
            # 디버깅 원하면 여기서 print(e, key) 찍어봐도 됨
            pass

    print("가중치 로드 완료!")

In [6]:
from unet_v2 import UNet
CHANNELS = [1, 16, 32, 64, 128, 256]
FILTER_SIZE = 3 
model = UNet(CHANNELS, filter_size=3)
w_before = model.enc1.conv1.W.clone()

load_custom_weights(model, CHECKPOINT_PATH)

w_after = model.enc1.conv1.W
print(torch.mean(torch.abs(w_after - w_before)))

가중치 파일 로드 중: ./checkpoints_2/last_checkpoint.pth
가중치 로드 완료!
tensor(0.4577)


/var/folders/d6/klnztyj51h5d3y23pmnjp7v00000gn/T/ipykernel_67839/4124507608.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_